## Introduction, A conversation with Andrew Ng
* 큰 데이터셋 사용 시 over-fitting 위험이 조금 줄어듦
* Data augmentation 방법으로도 over-fitting 방지 가능
* Multicast learning (more than two classes)

## A conversation with Andrew Ng
더러운 데이터 (ex. 길이가 0인 데이터, 고양이를 들고 있는 사람 등) --> Tensorflow로 처리 가능!

## Training with the cats vs. dogs dataset

In [ ]:
# ImageDataGenerator : auto labeling subdirectories' dataset

from tensorflow.keras.preprocessing.image
import ImageDataGenerator

# create instance, normalization
train_datagen = ImageDataGenerator(rescale=1./255) 

train_generator = train_datagen.flow_from_directory(train_dir, 
                                                    target_size=(150, 150),
                                                    batch_size=20, 
                                                    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)


# create model
model = tf.keras.model.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(150, 150, 3)), # ret_size:(148, 148)
    tf.keras.layers.MaxPooling2D(2, 2), # (74, 74)
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'), # (72, 72)
    tf.keras.layers.MaxPooling2D(2, 2), # (36, 36)
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'), # (34, 34)
    tf.keras.layers.MaxPooling2D(2, 2), # (17, 17)
    tf.keras.layers.Flatten(), # 18496 == 17 * 17 * 64
    tf.keras.layers.Dense(512, activation='relu'), # 512
    tf.keras.layers.Dense(1, activation='sigmoid') # 1
])


# compile model
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001), # RMSprop automates learning-rate tuning (lr)
              metrics=['acc'])

# train model
history = model.fit_generator(
    train_generator, 
    steps_for_epoch=100, # batch_size=20 * 100회 후에야 전체 학습 데이터 2000장 로드 완료
    epochs=15,
    validation_data=validation_generator,
    validation_steps=50, # batch_size=20 * 50회 후에야 전체 검증 데이터 1000장 로드 완료
    verbose=2
)

## Fixing through cropping
위의 방식처럼 학습을 진행하면 한두장 잘못 분류하는 경우가 생김 ==> 해결책? 
> cropping! (고양이 사진에서 헷갈리게 만들만한 영역을 자른다)

## Visualizing the effect of the convolutions
* 컨볼루션 신경망이 이미지의 어떤 특징을 중점으로 학습하게 되는지 visualize!
> ex. 강아지의 귀, 개구리의 눈 등
* model.layers (API) : allows to find the outputs and iterate through them, creating a visualization model for each one
> ```py
successive_outputs = [layer.output for layer in model.layers[1:]]
visualization_model = Model(img_input, successive_outputs)
```

## Looking at accuracy and loss
* cf. history = model.fit_generator() == model.fit()
* 데이터 수가 적으면 **overfitting** 일어남
> **overfitting** : train_acc는 점점 100%에 가까워지는데, val_acc는 70%까지만 초기에 오르고 더 이상 오르지 않음!